# Header

This code borrows extensively from https://github.com/sudhof/politeness/tree/python3

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import ipyparallel
import os
import sys
sys.path.append('/home/jwlock/research/reddit/CSSLabs_Community_Dynamics')
sys.path.append('/home/jwlock/research/reddit/CSSLabs_Community_Dynamics/politeness')

from politeness.scripts.format_input import format_doc
from politeness.model import score

c = ipyparallel.Client()
view = c.load_balanced_view()

/home/jwlock/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator SVC from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [7]:
def get_politeness(txt):
    fs = format_doc(txt)
    ps = []
    for f in fs:
        ps.append(score(f)['polite'])
    return np.mean(ps)

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [8]:
def process_file(job):
    import sys
    import pandas as pd
    import numpy as np
    from tqdm import tqdm
    
    sys.path.append('/home/jwlock/research/reddit/CSSLabs_Community_Dynamics')
    sys.path.append('/home/jwlock/research/reddit/CSSLabs_Community_Dynamics/politeness')

    from politeness.scripts.format_input import format_doc
    from politeness.model import score
    
    def get_politeness(txt):
        fs = format_doc(txt)
        ps = []
        for f in fs:
            ps.append(score(f)['polite'])
        return np.mean(ps)
    
    df = pd.read_csv(job['file'], sep='\t')
    with open('/home/jwlock/research/reddit/CSSLabs_Community_Dynamics/data/'+
              job['subreddit']+'.prog', 'w') as f:
        tqdm.pandas(file = f)
        df['politeness'] = df.body.progress_apply(get_politeness)
    
    df.to_csv('/home/jwlock/research/reddit/CSSLabs_Community_Dynamics/data/'+job['subreddit']+'.tsv', sep='\t', index=False)
    
    return

def process_df(df):
    import sys
    import pandas as pd
    import numpy as np
    from tqdm import tqdm
    
    sys.path.append('/home/jwlock/research/reddit/CSSLabs_Community_Dynamics')
    sys.path.append('/home/jwlock/research/reddit/CSSLabs_Community_Dynamics/politeness')

    from politeness.scripts.format_input import format_doc
    from politeness.model import score
    
    def get_politeness(txt):
        txt = str(txt)
        fs = format_doc(txt)
        ps = []
        for f in fs:
            ps.append(score(f)['polite'])
        return np.mean(ps)
    
    df['politeness'] = df.body.apply(get_politeness)
        
    return df

In [9]:
txt = "this is a text document with some words. I like words."
get_politeness(txt)

0.4113486466093339

In [10]:
files = os.listdir('../sampled')
done = os.listdir('data')
len(files)

32

In [11]:
jobs = []
for f in files:
    if f.endswith('tsv'):
        if f in done:
            print('already finished', f)
            pass
        else:
            print('adding', f)
            tmp = {}
            tmp['file'] = '/home/jwlock/research/reddit/sampled/'+f
            tmp['subreddit'] = f[:-4]
            jobs.append(tmp)
    
len(jobs)

adding demsocialist.tsv
adding puppies.tsv
adding republicans.tsv
adding CatGifs.tsv
adding cats.tsv
adding GreenParty.tsv
adding msu.tsv
adding StartledCats.tsv
adding uofm.tsv
adding CatsStandingUp.tsv
adding dogpictures.tsv
adding OSU.tsv
adding communism.tsv
adding TrollXChromosomes_short.tsv
adding democrats.tsv
adding Liberal.tsv
adding dogs_short.tsv
adding socialism_short.tsv
adding TwoXChromosomes_short.tsv
adding Libertarian_short.tsv
adding progressive.tsv
adding FULLCOMMUNISM.tsv
adding Republican.tsv
adding Dogtraining.tsv
adding NeutralPolitics.tsv
adding socialism.tsv
adding dogs.tsv
adding TrollXChromosomes.tsv
adding Libertarian.tsv
adding TwoXChromosomes.tsv


In [13]:
for j in jobs:
    print('Working on', j['subreddit'])
    df = pd.read_csv(j['file'], sep='\t')
    chunks = chunker(df, 100)
    result = view.map_async(process_df, chunks)
    result.wait_interactive()
    df = pd.concat(result)
    df.to_csv('data/'+j['subreddit']+'.tsv', sep='\t', index=False)

   0/55 tasks finished after   19 s

KeyboardInterrupt: 